In [1]:
from model import FreckerDataSet
from torch.utils.data import DataLoader, random_split
import torch
import torch.optim as optim
import torch.nn.functional as F
from model import MaskLoss, FreckersNet
from torch.optim.lr_scheduler import OneCycleLR

In [2]:
# file = r"C:\Users\lucyc\Desktop\freckers_data\P1.h5"
# file2 = r"C:\Users\lucyc\Desktop\freckers_data\P2.h5"
# dataset1 = FreckerDataSet(file_path=file)
# dataset2 = FreckerDataSet(file_path=file2)

#dataset = torch.utils.data.ConcatDataset([dataset1, dataset2])

file_base_path = r"C:\Users\lucyc\Desktop\data"
versions = [str(i) for i in range(1, 2)]
datasets = [FreckerDataSet(file_base_path + "\\" + v + ".h5") for v in versions]

# # ÂêàÂπ∂‰∏§‰∏™Êï∞ÊçÆÈõÜ
dataset = torch.utils.data.ConcatDataset(datasets)

# dataset = FreckerDataSet(file_path=file)
# dataset = torch.utils.data.Subset(dataset, range(int(len(dataset) * (9/11)), len(dataset)))


# ÊåâÁÖßÈ°∫Â∫èÂàÜÂâ≤Êï∞ÊçÆÈõÜ
train_size = int(0.8 * len(dataset))  # 50% Áî®‰∫éËÆ≠ÁªÉ
val_size = len(dataset) - train_size  # 50% Áî®‰∫éÈ™åËØÅ

# ‰ΩøÁî® Subset ÂàÜÂâ≤Êï∞ÊçÆÈõÜ
train_dataset = torch.utils.data.Subset(dataset, range(train_size))
val_dataset = torch.utils.data.Subset(dataset, range(train_size, len(dataset)))

train_dataset, _ = random_split(train_dataset, [int(0.005 * len(train_dataset)), len(train_dataset) - int(0.1 * len(train_dataset))])
val_dataset, _ = random_split(val_dataset, [int(0.1 * len(val_dataset)), len(val_dataset) - int(0.1 * len(val_dataset))])
# ÈöèÊú∫ÂàÜÂâ≤
# train_dataset, val_dataset = random_split(dataset, [int(0.5 * len(dataset)), len(dataset) - int(0.5 * len(dataset))])
# print(f"ÂêàÂπ∂ÂêéÁöÑÊï∞ÊçÆÈõÜÂ§ßÂ∞è: {len(train_dataset)}")

# # ÂÆö‰πâËÆ≠ÁªÉÈõÜÂíåÈ™åËØÅÈõÜÁöÑÂ§ßÂ∞è
# train_size = int(0.8 * len(dataset))  # 80% Áî®‰∫éËÆ≠ÁªÉ
# val_size = len(dataset) - train_size  # 20% Áî®‰∫éÈ™åËØÅ

# # ‰ΩøÁî® random_split ÂàÜÂâ≤Êï∞ÊçÆÈõÜ
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

batch_size = 64
shuffle = True
num_workers = 1

# ÂàõÂª∫ DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle)

In [6]:
len(train_dataset)

21096

In [3]:
model = FreckersNet()

In [9]:
len(dataset)

80053

In [4]:
num_epochs = 10
stop_epoch = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.1)
# scheduler = OneCycleLR(
#     optimizer,
#     max_lr=0.0001,  # ÊúÄÂ§ßÂ≠¶‰π†Áéá
#     steps_per_epoch=len(train_loader),
#     epochs=num_epochs,
#     pct_start=0.4  # warmup ÁöÑÊØî‰æã
# )
mask_loss = MaskLoss()

train_loss_record = []
eval_loss_record = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for gameboard, action_prob, value in train_loader:
        gameboard = gameboard.to(device)
        action_prob = action_prob.to(device)
        value = value.to(device)

        optimizer.zero_grad()
        p_action_prob, p_value = model(gameboard)
        # ËÆ°ÁÆóÂèåÊçüÂ§±
        loss_img = mask_loss(p_action_prob, action_prob)
        loss_prob = F.mse_loss(p_value.view(-1), value)
        #loss_prob = 0
        total_loss = loss_img + loss_prob
        
        total_loss.backward()
        optimizer.step()
        #scheduler.step()
        train_loss += total_loss.item()
    
    train_loss /= len(train_loader)
    train_loss_record.append(train_loss)

    model.eval()
    eval_loss = 0.0
    with torch.no_grad():
        for gameboard, action_prob, value in val_loader:
            gameboard = gameboard.to(device)
            action_prob = action_prob.to(device)
            value = value.to(device)

            p_action_prob, p_value = model(gameboard)
            loss_img = mask_loss(p_action_prob, action_prob)
            loss_prob = F.mse_loss(p_value.view(-1), value)
            #loss_prob = 0
            total_loss = loss_img + loss_prob
            
            eval_loss += total_loss.item()
    
    eval_loss /= len(val_loader)
    eval_loss_record.append(eval_loss)

    print(f"Epoch {epoch+1}, Training Loss: {train_loss:.8f}, Validation Loss: {eval_loss:.8f}")

    if epoch == stop_epoch:
        torch.save(model, "C:\\Users\\lucyc\\Desktop\\s2.pth")



c:\Users\lucyc\Desktop\freckers_zero\freckers\model.py:149: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_softmax = F.log_softmax(pred_main_flat[i][main_mask_flat[i]])


Epoch 1, Training Loss: 1.18147762, Validation Loss: 1.19928638
Epoch 2, Training Loss: 1.13789078, Validation Loss: 1.16522969
Epoch 3, Training Loss: 1.12382387, Validation Loss: 1.12652934
Epoch 4, Training Loss: 1.11336057, Validation Loss: 1.11317057
Epoch 5, Training Loss: 1.10304775, Validation Loss: 1.10785724
Epoch 6, Training Loss: 1.09711702, Validation Loss: 1.11595441
Epoch 7, Training Loss: 1.09567774, Validation Loss: 1.11190812
Epoch 8, Training Loss: 1.09039742, Validation Loss: 1.09515086
Epoch 9, Training Loss: 1.08493705, Validation Loss: 1.11511261
Epoch 10, Training Loss: 1.08404271, Validation Loss: 1.11211471


In [5]:
import matplotlib.pyplot as plt

# ÁªòÂà∂ËÆ≠ÁªÉÊçüÂ§±ÂíåÈ™åËØÅÊçüÂ§±ÁöÑÂØπÊØîÂõæ
plt.figure(figsize=(10, 5))
plt.plot(train_loss_record, label='Train Loss')
plt.plot(eval_loss_record, label='Eval Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Train Loss vs Eval Loss')
plt.legend()
plt.show()


In [18]:
import torch
from fnet import MaskLoss, Conv3DStack
model = torch.load("C:\\Users\\lucyc\\Desktop\\freckers_data\\P2_Mini.pth", weights_only=False)
model2 = torch.load("C:\\Users\\lucyc\\Desktop\\freckers_data\\P1_Mini.pth", weights_only=False)
#model2 = Conv3DStack()

In [20]:
from mcts import mcts_data_collect, mcts_competition
class MctsConfig:
    def __init__(self) -> None:
        self.c = 1.5
        self.t = 1
        self.finish = False
        self.visulze = False
        self.small = 0.0000001

        self.dirichlet_alpha = 0.03
        self.dirichlet_epsilon = 0.25

#res1 = mcts_competition(model, 1, r"C:\Users\lucyc\Desktop\freckers_data\temp.h5", MctsConfig(), 3, 150, model2)
print(mcts_competition(model2, 1, r"C:\Users\lucyc\Desktop\freckers_data\temp.h5", MctsConfig(), 3, 150, model, visulze=True))
print(mcts_competition(model2, 1, r"C:\Users\lucyc\Desktop\freckers_data\temp.h5", MctsConfig(), 3, 150, model, visulze=False))

Á∫øÁ®ã 1 Á¨¨ 0 ËΩÆÊ∏∏Êàè  Á¨¨ 0 Ê≠• Ê®°ÊãüËøõË°å‰∏≠

Rounds:  1
üü¢üî¥üî¥üî¥üî¥üî¥üî¥üü¢
‚ö™üü¢üü¢üü¢üü¢üü¢üü¢‚ö™
‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™
‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™
‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™
‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™
‚ö™üü¢üü¢üü¢üü¢üü¢üü¢‚ö™
üîµ‚ö™üîµüîµüîµüîµüîµüü¢
move action:  (7, 1, 7, 0, False)
Á∫øÁ®ã 1 Á¨¨ 0 ËΩÆÊ∏∏Êàè  Á¨¨ 1 Ê≠• Ê®°ÊãüËøõË°å‰∏≠

Rounds:  2
üî¥üî¥‚ö™üî¥üî¥üî¥üî¥üü¢
‚ö™üü¢üü¢üü¢üü¢üü¢üü¢‚ö™
‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™
‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™
‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™
‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™
‚ö™üü¢üü¢üü¢üü¢üü¢üü¢‚ö™
üîµ‚ö™üîµüîµüîµüîµüîµüü¢
move action:  (0, 2, 0, 0, False)
Á∫øÁ®ã 1 Á¨¨ 0 ËΩÆÊ∏∏Êàè  Á¨¨ 2 Ê≠• Ê®°ÊãüËøõË°å‰∏≠

Rounds:  3
üî¥üî¥‚ö™üî¥üî¥üî¥üî¥üü¢
‚ö™üü¢üü¢üü¢üü¢üü¢üü¢‚ö™
‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™
‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™
‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™
‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™‚ö™
‚ö™üü¢üü¢üîµüü¢üü¢üü¢‚ö™
üîµ‚ö™üîµüîµ‚ö™üîµüîµüü¢
move action:  (7, 4, 6, 3, False)
Á∫øÁ®ã 1 Á¨¨ 0 ËΩÆÊ∏∏Êàè  Á¨

KeyboardInterrupt: 

In [ ]:
import threading
import time
from mcts import mcts_data_collect

#(model, thread_num, file, rounds=100, sim_step=300)
# ÂàõÂª∫Â§ö‰∏™Á∫øÁ®ã
threads = []
for i in range(8):
    thread = threading.Thread(target=mcts_data_collect, 
            args=(model, i+1, f"C:\\Users\\lucyc\\Desktop\\freckers_data\\batch_1.h5", 30, 500))
    threads.append(thread)
    thread.start()

# Á≠âÂæÖÊâÄÊúâÁ∫øÁ®ãÂÆåÊàê
for thread in threads:
    thread.join()


In [12]:
torch.save(model, "C:\\Users\\lucyc\\Desktop\\freckers_data\\batch_2_no_inverse.pth")

In [9]:
data.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss_rec'])

In [29]:
import numpy as np

In [68]:
import torch

In [69]:
data = torch.load(r"C:\Users\lucyc\Desktop\models\68.pth")
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize=(10, 5))
plt.plot(data["loss_rec"], label='Training Loss')

# ËÆ°ÁÆóÂπ∂ÁªòÂà∂ÁßªÂä®Âπ≥ÂùáÁ∫ø
window_size = 10  # ËÆæÁΩÆÁßªÂä®Âπ≥ÂùáÁ™óÂè£Â§ßÂ∞è
moving_avg = np.convolve(data["loss_rec"], np.ones(window_size)/window_size, mode='valid')
plt.plot(range(window_size-1, len(data["loss_rec"])), moving_avg, label=f'Moving Average (Window={window_size})', color='red')

plt.title('Training Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [169]:
import torch
import torch.nn.functional as F

In [204]:
predictions = torch.randn(5, 65, 8, 8)
targets = torch.zeros(5, 65, 8, 8)
random_indices = torch.randperm(5*65*8*8)[:2100]
targets.view(-1)[random_indices] = torch.rand(2100)

In [205]:
# Assuming predictions and targets are already PyTorch tensors
# If not, convert them:
# predictions = torch.from_numpy(predictions)
# targets = torch.from_numpy(targets)

# Split predictions and targets
pred_main = predictions[:, :-1]  # (b, 64, 8, 8)
pred_grow = predictions[:, -1]   # (b, 8, 8)
label_main = targets[:, :-1]
label_grow = targets[:, -1]

# Grow mask operations
grow_mask = label_grow != 0
label_grow = label_grow * grow_mask
pred_grow = pred_grow * grow_mask

# Count non-zero elements and sums
label_grow_counts = torch.count_nonzero(label_grow, dim=(1, 2))
label_grow_sums = torch.sum(label_grow, dim=(1, 2))
pred_grow_counts = torch.count_nonzero(pred_grow, dim=(1, 2))
pred_grow_sums = torch.sum(pred_grow, dim=(1, 2))

# Calculate means with zero division handling
label_grow_mean = torch.zeros_like(label_grow_sums)
mask = label_grow_counts != 0
label_grow_mean[mask] = label_grow_sums[mask] / label_grow_counts[mask]

pred_grow_mean = torch.zeros_like(pred_grow_sums)
mask = pred_grow_counts != 0
pred_grow_mean[mask] = pred_grow_sums[mask] / pred_grow_counts[mask]

# Main mask operations
main_mask = label_main != 0
label_main = label_main * main_mask
pred_main = pred_main * main_mask

In [228]:

B, C, H, W = pred_main.shape
main_mask_flat = main_mask.view(B, -1)
main_mask_flat = torch.concat((main_mask_flat, torch.ones(B, 1, dtype=torch.bool)), dim=1)
pred_main_flat = pred_main.view(B, -1)  # -1 Ëá™Âä®ËÆ°ÁÆó H*W
pred_main_flat = torch.concat((pred_main_flat, pred_grow_mean.view(B, 1)), dim=1)
label_main_flat = label_main.view(B, -1)  # -1 Ëá™Âä®ËÆ°ÁÆó H*W
label_main_flat = torch.concat((label_main_flat, label_grow_mean.view(B, 1)), dim=1)

loss = 0
count = 0
for i in range(B):
    log_softmax = F.log_softmax(pred_main_flat[i][main_mask_flat[i]])
    labels = label_main_flat[i][main_mask_flat[i]]
    loss += - (labels * log_softmax).sum()  # (B, 8, 8)
    count += len(labels)

loss/count


# pred_main_softmax = torch.stack(pred_softmax)
# labels = torch.stack(labels)

# cross_entropy = - (labels * pred_main_softmax).sum(dim=1)  # (B, 8, 8)





C:\Users\lucyc\AppData\Local\Temp\ipykernel_40920\1087618014.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  log_softmax = F.log_softmax(pred_main_flat[i][main_mask_flat[i]])


tensor(3.2205)

In [208]:
pred_main_softmax.shape

torch.Size([5, 4097])

In [177]:
label_main_flat.shape


torch.Size([5, 4097])

In [159]:
pred_grow_mean.view(5, 1).shape

torch.Size([5, 1])

In [161]:
pred_main_flat.shape

torch.Size([5, 4096])

In [160]:
torch.concat((pred_main_flat, pred_grow_mean.view(5, 1)), dim=1).shape

torch.Size([5, 4097])

In [102]:
import torch.nn.functional as F

In [93]:
predictions = np.random.randn(5, 65,8,8)
targets = np.zeros((5,65,8,8))
targets = np.zeros((5, 65, 8, 8))
random_indices = np.random.choice(np.arange(5*65*8*8), size=2100, replace=False)
np.put(targets, random_indices, np.random.rand(1))


In [94]:
pred_main = predictions[:, :-1]  # (b, 64, 8, 8)
pred_grow = predictions[:, -1]   # (b, 8, 8)
label_main = targets[:, :-1]
label_grow = targets[:, -1]

In [95]:
grow_mask = label_grow != 0
label_grow = label_grow * grow_mask
pred_grow = pred_grow * grow_mask

label_grow_counts = np.count_nonzero(label_grow, axis=(1, 2))
label_grow_sums = np.sum(label_grow, axis=(1, 2))
pred_grow_counts = np.count_nonzero(pred_grow, axis=(1, 2))
pred_grow_sums = np.sum(pred_grow, axis=(1, 2))

label_grow_mean = np.divide(label_grow_sums, label_grow_counts, out=np.zeros_like(label_grow_sums), where=label_grow_counts!=0)
pred_grow_mean = np.divide(pred_grow_sums, pred_grow_counts, out=np.zeros_like(pred_grow_sums), where=pred_grow_counts!=0)


In [97]:
main_mask = label_main != 0
label_main = label_main * main_mask
pred_main = pred_main * main_mask


In [105]:
label_main

array([[[[0.        , 0.        , 0.        , ..., 0.        ,
          0.99874407, 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.99874407],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.99874407, 0.99874407, ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.99874407, ..., 0.        ,
          0.        , 0.99874407],
         [0.99874407, 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]],

        [[0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.99874407, 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0. 

In [11]:
from model import FreckerDataSet
from torch.utils.data import DataLoader, random_split
import torch
import torch.optim as optim
import torch.nn.functional as F
from model import MaskLoss, FreckersNet
from torch.optim.lr_scheduler import OneCycleLR


In [12]:
file_base_path = r"C:\Users\lucyc\Desktop\data\1.h5"

datasets = FreckerDataSet(file_base_path)


In [17]:
import numpy as np

In [13]:
datasets[0][0].shape


torch.Size([16, 8, 8])

In [14]:
datasets[0][1].shape


torch.Size([65, 8, 8])

In [15]:
datasets[0][2]


tensor(1.)

In [18]:
# Â∞ÜÊï∞ÊçÆËΩ¨Êç¢‰∏∫numpyÊï∞ÁªÑ
input_data = np.array([datasets[i][0].numpy() for i in range(len(datasets))])
labels = np.array([datasets[i][2] for i in range(len(datasets))])
current_player = input_data[:, 15, 0, 0]  # Á¨¨15Â±ÇÁ¨¨0Ë°åÁ¨¨0Âàó

# ‰ΩøÁî®ÂêëÈáèÂåñÊìç‰ΩúËÆ°ÁÆóÊù°‰ª∂
p1_layer = input_data[:, 12, 7, :]  # Á¨¨12Â±ÇÁ¨¨7Ë°å
p2_layer = input_data[:, 13, 0, :]  # Á¨¨13Â±ÇÁ¨¨0Ë°å

# ËÆ°ÁÆóËÉúÂà©Êù°‰ª∂
p1_sum_cond = p1_layer.sum(axis=1) == 5
p2_sum_cond = p2_layer.sum(axis=1) == 5

# ÁªÑÂêàÊâÄÊúâÊù°‰ª∂
p1_win = np.sum((p1_sum_cond) & (labels == 1) & (current_player == 0))
p2_win = np.sum((p2_sum_cond) & (labels == 1) & (current_player == 1))

print(f"\n\nP1 Win {p1_win}")
print(f"P2 Win {p2_win}")





P1 Win 5908
P2 Win 5519


In [19]:
import numpy as np

# Â∞ÜÂâç100000‰∏™Ê†∑Êú¨ÁöÑËæìÂÖ•Êï∞ÊçÆËΩ¨Êç¢‰∏∫numpyÊï∞ÁªÑ
input_data = np.array([datasets[i][0].numpy() for i in range(len(datasets))])

# ‰ΩøÁî®numpyÂêëÈáèÂåñÊìç‰ΩúËÆ°ÁÆóËÉúÂà©Êù°‰ª∂
p1_condition = input_data[:, 12, 7, :].sum(axis=1) == 5
p2_condition = input_data[:, 13, 0, :].sum(axis=1) == 5

p1_win = np.sum(p1_condition)
p2_win = np.sum(p2_condition)

print(f"\n\nP1 Win {p1_win}")
print(f"P2 Win {p2_win}")




P1 Win 14444
P2 Win 13566


In [70]:
step = """
end within 89 steps
player: 0, Win: 1
end within 111 steps
player: 0, Win: 1
end within 99 steps
player: 0, Win: 1
end within 109 steps
player: 0, Win: 1
end within 103 steps
player: 0, Win: 1
end within 92 steps
player: 1, Win: 1
end within 94 steps
player: 1, Win: 1
end within 121 steps
player: 0, Win: 1
end within 105 steps
player: 0, Win: 1
end within 93 steps
player: 0, Win: 1
end within 97 steps
player: 0, Win: 1
end within 102 steps
player: 1, Win: 1
end within 107 steps
player: 0, Win: 1
end within 95 steps
player: 0, Win: 1
end within 94 steps
player: 1, Win: 1
end within 103 steps
player: 0, Win: 1
end within 99 steps
player: 0, Win: 1
end within 90 steps
player: 1, Win: 1
end within 76 steps
player: 1, Win: 1
end within 110 steps
player: 1, Win: 1
end within 102 steps
player: 1, Win: 1
end within 92 steps
player: 1, Win: 1
end within 103 steps
player: 0, Win: 1
end within 96 steps
player: 1, Win: 1
"""

In [71]:
import re

def calculate_average_steps(input_text):
    # ‰ΩøÁî®Ê≠£ÂàôË°®ËææÂºèÂåπÈÖçÊâÄÊúâ"end within X steps"‰∏≠ÁöÑX
    steps = re.findall(r'end within (\d+) steps', input_text)
    
    # Â∞ÜÂåπÈÖçÂà∞ÁöÑÂ≠óÁ¨¶‰∏≤ËΩ¨Êç¢‰∏∫Êï¥Êï∞
    steps = [int(step) for step in steps]
    
    # ËÆ°ÁÆóÂùáÂÄº
    if steps:
        average = sum(steps) / len(steps)
        return average
    else:
        return 0

calculate_average_steps(step)

99.25